In [1]:
import pandas as pd

In [2]:
def get_sp_500_symbols():
    sp_500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

    tables = pd.read_html(sp_500_url)
    sp_500 = tables[0]
    return sp_500['Symbol'].tolist()
get_sp_500_symbols()

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',


In [3]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [4]:
import yfinance as yf

In [5]:
def get_finance_data(symbols,start_date,end_date,interval):
    result={}
    for symbol in symbols:
        data = yf.download(tickers=symbol,start=start_date,end=end_date,interval=interval)
        if not data.empty:
            result[symbol]=data
    return result
get_finance_data('AAPL','2025-04-11','2025-04-18','1d')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['P']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-04-11 -> 2025-04-18)')
[*********************100%***********************]  1 of 1 completed


{'A': Price            Close        High         Low        Open   Volume
 Ticker               A           A           A           A        A
 Date                                                               
 2025-04-11  102.709999  103.339996   98.860001  101.150002  2849100
 2025-04-14  105.190002  106.209999  104.000000  104.940002  1906700
 2025-04-15  103.120003  105.250000  102.540001  104.300003  1326700
 2025-04-16  102.699997  104.830002  101.870003  103.220001  1454400
 2025-04-17  102.480003  103.500000  101.360001  102.629997  1769900,
 'L': Price           Close       High        Low       Open   Volume
 Ticker              L          L          L          L        L
 Date                                                           
 2025-04-11  84.680000  85.160004  83.480003  84.470001  1386400
 2025-04-14  86.320000  87.250000  85.389999  85.820000   686400
 2025-04-15  86.129997  87.220001  86.099998  86.769997   446100
 2025-04-16  85.199997  86.790001  84.639999  8

In [6]:
def transform_data(data, symbol):
    data.columns = [col[0] for col in data.columns]
    data = data.reset_index()
    print(data)

    data = data[data["Close"] != 0]
    data["symbol"] = symbol
    data['close_change'] = data['Close'].diff().fillna(0)
    data['close_pct_change'] = data['Close'].pct_change().fillna(0) * 100
    print(data)
    return data[['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'symbol', 'close_change', 'close_pct_change']]



In [14]:

def ingest_yfinance_data(symbol_data, final_table, interval):
    values = []
    for symbol in symbol_data:
        try:
            start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = datetime.now().strftime('%Y-%m-%d')
            data_dict = get_finance_data([symbol], start_date, end_date, interval)
            if symbol in data_dict:
                data = transform_data(data_dict[symbol], symbol)
    
                if not data.empty:
                    values.extend([tuple(x) for x in data.to_numpy()])
        except Exception as e:
            print(e)
            
            
            
            error_logger.error(f"{symbol} transformation error: {str(e)}")
    if values:
        insert_data_to_db(final_table, values)
ingest_yfinance_data(['AAPL','MSFT'],"",'1d')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


        Date       Close        High         Low        Open    Volume
0 2025-04-21  193.160004  193.800003  189.809998  193.270004  46742500
1 2025-04-22  199.740005  201.589996  195.970001  196.119995  52976400
2 2025-04-23  204.600006  208.000000  202.800003  206.000000  52929200
3 2025-04-24  208.369995  208.830002  202.940002  204.889999  47311000
4 2025-04-25  209.279999  209.750000  206.199997  206.369995  38198600
        Date       Close        High         Low        Open    Volume symbol  \
0 2025-04-21  193.160004  193.800003  189.809998  193.270004  46742500   AAPL   
1 2025-04-22  199.740005  201.589996  195.970001  196.119995  52976400   AAPL   
2 2025-04-23  204.600006  208.000000  202.800003  206.000000  52929200   AAPL   
3 2025-04-24  208.369995  208.830002  202.940002  204.889999  47311000   AAPL   
4 2025-04-25  209.279999  209.750000  206.199997  206.369995  38198600   AAPL   

   close_change  close_pct_change  
0      0.000000          0.000000  
1      6.580002

In [8]:
from datetime import datetime,timedelta


In [9]:
import logging

In [17]:
def create_connection():
    conn_str = (
          r"DRIVER={SQL Server};"
        r"SERVER=DESKTOP-KDIPAH2;"  
        r"DATABASE=stock;"  
        r"Trusted_Connection=yes;"
        r"UID=sa;"
        r"PWD=taha.bilalpass;"    
    )

    try:
        conn = pyodbc.connect(conn_str)
        print("✅ Connected to SQL Server successfully!")
        return conn
    except Exception as e:
        print(f"❌ Error connecting to SQL Server: {e}")
        return None

def create_table_if_not_exists(table):
    conn = create_connection()
    if not conn:
        return

    cursor = conn.cursor()
    create_table_query = """
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='stock_prices' and xtype='U')
    CREATE TABLE stock_prices (
        trade_date DATETIME,
        close_price FLOAT,
        high_price FLOAT,
        low_price FLOAT,
        open_price FLOAT,
        volume BIGINT,
        symbol VARCHAR(10),
        close_change FLOAT,
        close_pct_change FLOAT
    )
    """
    try:
        cursor.execute(create_table_query)
        conn.commit()
        print("✅ Table ensured: stock_prices")
    except Exception as e:
        print(f"❌ Error creating table: {e}")
    finally:
        cursor.close()
        conn.close()


def insert_data_to_db(table_name, values):
    conn = create_connection()
    if not conn:
        return

    cursor = conn.cursor()
    insert_query = f"""
    INSERT INTO {table_name} (
        trade_date, close_price, high_price, low_price, open_price,
        volume, symbol, close_change, close_pct_change
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"""

    try:
        cursor.executemany(insert_query, values)
        conn.commit()
        print(f"✅ Inserted {len(values)} rows into {table_name}")
    except Exception as e:
        print(f"❌ Error inserting data: {e}")
    finally:
        cursor.close()
        conn.close()

In [13]:
import pyodbc

In [15]:

def get_sp500_symbols():
    sp_500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    
    tables = pd.read_html(sp_500_url)
    sp_500 = tables[0]
    return sp_500['Symbol'].tolist()

def get_finance_data (symbols,start_date,end_date, interval):
    result ={}
    for symbol in symbols:
        data = yf.download(tickers = symbol,start=start_date,end = end_date, interval=interval )
        if not data.empty:
            result[symbol] = data
    return result 
    
def transform_data(data, symbol):
    data.columns = [col[0] for col in data.columns]
    data = data.reset_index()

    # data = data[data["Close"] != 0]
    data["symbol"] = symbol
    data['close_change'] = data['Close'].diff().fillna(0)
    data['close_pct_change'] = data['Close'].pct_change().fillna(0) * 100
    return data[['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'symbol', 'close_change', 'close_pct_change']]

def ingest_yfinance_data(symbol_data, table_name, interval):
    values = []
    for symbol in symbol_data:
        try:
            start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = datetime.now().strftime('%Y-%m-%d')
            data_dict = get_finance_data([symbol], start_date, end_date, interval)
            if symbol in data_dict:
                data = transform_data(data_dict[symbol], symbol)
                if not data.empty:
                    values.extend([tuple(x) for x in data.to_numpy()])
        except Exception as e:
            print(e)
            # error_logger.error(f"{symbol} transformation error: {str(e)}")
    if values:
        insert_data_to_db(table_name, values)


def main(interval):
    table_name = "stock_prices"
    symbols = get_sp500_symbols()
    print(len(symbols))
    create_table_if_not_exists(table_name)
    ingest_yfinance_data(symbols ,table_name, interval)
    
    

In [18]:
main('1d')

503
✅ Connected to SQL Server successfully!
✅ Table ensured: stock_prices


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Failed to get ticker 'AKAM' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
[*********************100%************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

✅ Connected to SQL Server successfully!
✅ Inserted 2505 rows into stock_prices
